## Record Linkage

In [ ]:
!pip install recordlinkage --q

In [1]:
import numpy as np 
import pandas as pd
import recordlinkage

In [2]:
final = pd.read_csv("src/final_dataset.csv")
final = final.drop(columns=['Unnamed: 0'])
final.head()

,name,country,sector,founded,marketcap,revenue,employees,links,ceo
0,LACEWORK,UNITED STATES,CYBERSECURITY,2015.0,NaN,NaN,NaN,NaN,NaN
1,TIPALTI,UNITED STATES,FINTECH,2010.0,NaN,NaN,NaN,NaN,NaN
2,TEMPUS,UNITED STATES,HEALTH,2015.0,NaN,NaN,NaN,NaN,NaN
3,ANDURIL,UNITED STATES,ARTIFICIAL INTELLIGENCE,2017.0,NaN,NaN,NaN,NaN,NaN
4,BOLT,ESTONIA,AUTO & TRANSPORTATION,2013.0,NaN,NaN,NaN,NaN,NaN


In [3]:
final = final.sort_values('name')
final = final.reset_index(drop=True)
final.head()

,name,country,sector,founded,marketcap,revenue,employees,links,ceo
0,ADDUS HOMECARE,USA,NaN,NaN,1.690000e+09,NaN,NaN,NaN,NaN
1,#SINOB,GERMANY,RETAIL,2015.0,NaN,4.0,14.0,HTTPS://WWW.SINOB.DE/,NaN
2,(DIA) DISTRIBUIDORA INTERNACIONAL DE ALIMENTACIN,SPAIN,NaN,NaN,8.700000e+08,NaN,NaN,NaN,NaN
3,(DIA) DISTRIBUIDORA INTERNACIONAL DE ALIMENTACIÓN,SPAIN,NaN,NaN,8.700000e+08,NaN,NaN,NaN,NaN
4,(DIA) DISTRIBUIDORA INTERNACIONAL DE ALIMENTACIÓN,SPAIN,NaN,NaN,8.200000e+08,NaN,NaN,NaN,NaN


In [80]:
final.to_csv("./src/sorted_final_dataset.csv")

In [4]:
final.isnull().sum()

name              0
country        6933
sector       122786
founded      132943
marketcap     98028
revenue      121663
employees    130431
links        156234
ceo          151873
dtype: int64

In [ ]:
for column_name in final.columns:
    column = final[column_name]
    # Get the count of Zeros in column 
    count = (column == 0.0).sum()
    print('Count of zeros in column', column_name, ' is: ', count)

In [ ]:
for c in final.columns:
    print(final[c].dtype)
    if final[c].dtype == float:
        final[c] = final[c].replace(np.nan, 0)

In [ ]:
final.isnull().sum()

In [ ]:
final.info()

## Index

In [55]:
from difflib import SequenceMatcher

def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

In [53]:
from recordlinkage.base import BaseIndexAlgorithm
import time
class CustomIndex(BaseIndexAlgorithm):
    def _link_index(self, df_a, df_b):
        t0 = time.time()
        print(len(df_a))
        indici1=[]
        indici2=[]
        x = 100
        for i in range(0, len(df_a)):
            if(i%5000 == 0):
                print(i)
            if (i + x) >= len(df_a):
                    x -= 1
            for j in range(i, (i+x)):
                if(similar(df_a.loc[i, 'name'], df_a.loc[j, 'name'])>0.35):
                    indici1.append(i)
                    indici2.append(j)
        indici = [indici1, indici2]
        t1 = time.time()
        print(t1-t0)
        return pd.MultiIndex.from_arrays(indici, names=('first', 'second'))  

In [56]:
indexer = CustomIndex()
candidate_pairs = indexer.index(final, final)
pairs = candidate_pairs

173033
0
5000
10000
15000
20000
25000
30000
35000
40000
45000
50000
55000
60000
65000
70000
75000
80000
85000
90000
95000
100000
105000
110000
115000
120000
125000
130000
135000
140000
145000
150000
155000
160000
165000
170000
954.7259268760681


In [57]:
pairs

MultiIndex([(     0,      0),
            (     0,     19),
            (     0,     93),
            (     1,      1),
            (     2,      2),
            (     2,      3),
            (     2,      4),
            (     2,      5),
            (     2,      6),
            (     2,      7),
            ...
            (173028, 173028),
            (173028, 173029),
            (173028, 173030),
            (173028, 173031),
            (173029, 173029),
            (173029, 173030),
            (173029, 173031),
            (173030, 173030),
            (173030, 173031),
            (173031, 173031)],
           length=9387042)

In [58]:
# 80% training set, 20% test set
n_perc = int((len(pairs) * 80) / 100)
train_pairs = pairs[:n_perc]
test_pairs = pairs[n_perc:]

## Training 

In [59]:
from recordlinkage.base import BaseCompareFeature
class CompareNumbers(BaseCompareFeature):
    # Similarity score between two numbers
    def _compute_vectorized(self, s1, s2):
        s1 = abs(s1)
        s2 = abs(s2)
        return round(1 - abs(s1 - s2) / (s1 + s2))

In [70]:
# jarowinkler gives priority to the begining of the string
# levenshtein cares more about the order
# compare.string -> default: levenshtein
compare = recordlinkage.Compare()

# add missing_value=pd.NA in compare.string if founded, marketcap, revenue, employees added
compare.string('name', 'name', label="name", threshold=0.60)
compare.string('country', 'country', label="country", threshold=0.60, missing_value=np.nan)
compare.string('sector', 'sector', label="sector", threshold=0.50, missing_value=np.nan)
# compare.add(CompareNumbers('founded', 'founded', label="founded"))
# compare.add(CompareNumbers('marketcap', 'marketcap', label="marketcap"))
# compare.add(CompareNumbers('revenue', 'revenue', label="revenue"))
# compare.add(CompareNumbers('employees', 'employees', label="employees"))
compare.string('links', 'links', method='jarowinkler', label="links", threshold=0.50, missing_value=np.nan)
compare.string('ceo', 'ceo', method='jarowinkler', label="ceo", threshold=0.60, missing_value=np.nan)

training_features = compare.compute(train_pairs, final, final)
training_features['score'] = training_features.loc[:, 'name':'ceo'].sum(axis=1)
training_features['null_values'] = training_features.loc[:, 'name':'ceo'].isnull().sum(axis=1)

In [61]:
#training_features = training_features.fillna(0)

In [71]:
training_features.head(10)

name  country  sector  links  ceo  score  null_values
0 0    1.0      1.0     NaN    NaN  NaN    2.0            3
  19   0.0      0.0     NaN    NaN  NaN    0.0            3
  93   0.0      0.0     NaN    NaN  NaN    0.0            3
1 1    1.0      1.0     1.0    1.0  NaN    4.0            1
2 2    1.0      1.0     NaN    NaN  NaN    2.0            3
  3    1.0      1.0     NaN    NaN  NaN    2.0            3
  4    1.0      1.0     NaN    NaN  NaN    2.0            3
  5    1.0      1.0     NaN    NaN  NaN    2.0            3
  6    1.0      1.0     NaN    NaN  NaN    2.0            3
  7    1.0      1.0     NaN    NaN  NaN    2.0            3

for c in training_features.columns:
    training_features[c] = training_features[c].replace(np.nan, 0)

In [72]:
col = len(final.columns) - 4
training_matches = training_features[training_features['score'] >= (col - training_features['null_values'])/2]
matches_train = training_matches
training_matches = training_matches.reset_index()

In [73]:
# toDrop = ['name', 'country', 'sector', 'founded', 'marketcap', 'revenue', 'employees', 'links', 'ceo', 'score', 'null_values']
toDrop = ['name', 'country', 'sector', 'links', 'ceo', 'score', 'null_values']
training_matches = training_matches.drop(toDrop, axis=1)
training_matches.head()

,level_0,level_1
0,0,0
1,1,1
2,2,2
3,2,3
4,2,4


In [74]:
training_matches = pd.MultiIndex.from_frame(training_matches)

In [75]:
training_matches

MultiIndex([(     0,      0),
            (     1,      1),
            (     2,      2),
            (     2,      3),
            (     2,      4),
            (     2,      5),
            (     2,      6),
            (     2,      7),
            (     2,      8),
            (     3,      3),
            ...
            (138349, 138396),
            (138349, 138397),
            (138349, 138398),
            (138350, 138350),
            (138350, 138377),
            (138350, 138382),
            (138350, 138383),
            (138350, 138401),
            (138350, 138403),
            (138350, 138407)],
           names=['level_0', 'level_1'], length=2237168)

In [ ]:
def createDictionary(matches):
    d = {}
    ignoreSet = set()
    for k, v in matches:
        if k not in d.keys() and k not in ignoreSet:
            d[k] = [k]
            ignoreSet.add(k)
        elif k in d.keys() and v not in ignoreSet:
            d[k].append(v)
            ignoreSet.add(v)
    return d
            
dictionary = createDictionary(training_matches)
            

In [ ]:
dictionary

In [ ]:
def collapseMatches(df, dict_matches):
    df_collapsed = pd.DataFrame(columns=['name' , 'country', 'sector', 'founded', 'marketcap', 'revenue', 'employees', 'links', 'ceo'])
    indexesToDelete = []
    i = 0
    for k in dict_matches.keys():
        if(i%1000==0):
            print(i)
        i+=1
        indexesToDelete.append(k)
        row = df[df.index == k]
        for v in dict_matches[k]:
            indexesToDelete.append(v)
            for field in row:
                if(row[field].isnull().values.any()):
                    if(not df[df.index == v][field].isnull().values.any()):
                        row.at[k, field] = df[df.index==v][field].values[0]
        df_collapsed.loc[len(df_collapsed.index)]=row.values[0]
    return df_collapsed, indexesToDelete

df_collapsed_train, indexesToDeleteGlobal = collapseMatches(final, dictionary)
df_collapsed_train.to_csv('./src/train_matches_collapsed.csv', index=False)

In [96]:
df_collapsed_train.head(10)

,name,country,sector,founded,marketcap,revenue,employees,links,ceo
0,ADDUS HOMECARE,USA,NaN,NaN,1690000000.0,NaN,NaN,NaN,NaN
1,#SINOB,GERMANY,RETAIL,2015.0,NaN,4.0,14.0,HTTPS://WWW.SINOB.DE/,NaN
2,(DIA) DISTRIBUIDORA INTERNACIONAL DE ALIMENTACIN,SPAIN,SUPERMARKETS FOOD RETAIL,NaN,870000000.0,NaN,NaN,NaN,NaN
3,(HLBANK) HONG LEONG BANK,MALAYSIA,BANKS FINANCIAL-SERVICES,NaN,9720000000.0,NaN,NaN,NaN,NaN
4,.NET IT SYSTEMHAUS,GERMANY,TECHNOLOGY,2014.0,NaN,4280960.0,83.0,HTTPS://WWW.NET-IT-SYSTEMHAUS.COM,NaN
5,0044 LIMITED,UNITED KINGDOM,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,0X,UNITED STATES,FINTECH,2016.0,NaN,NaN,NaN,NaN,NaN
7,1 INDIA FAMILY MART,INDIA,RETAIL,2012.0,NaN,NaN,NaN,NaN,NaN
8,1&1,GERMANY,TELECOMMUNICATION INTERNET,1983.0,2430000000.0,4312000.0,3163.0,HTTPS://WWW.1UND1-DRILLISCH.DE,MR. RALPH DOMMERMUTH
9,1-800-FLOWERS,USA,[NAN],NaN,600000000.0,1860000000.0,4700.0,HTTPS://WWW.1800FLOWERS.COM/,NaN


In [84]:
training_features = training_features.fillna(0)
training_features.head()

name  country  sector  links  ceo  score
0 0    1.0      1.0     0.0    0.0  0.0    2.0
  19   0.0      0.0     0.0    0.0  0.0    0.0
  93   0.0      0.0     0.0    0.0  0.0    0.0
1 1    1.0      1.0     1.0    1.0  0.0    4.0
2 2    1.0      1.0     0.0    0.0  0.0    2.0

#### Classifier

In [85]:
classifier = recordlinkage.NaiveBayesClassifier()
classifier.fit(training_features, training_matches)

## Testing

In [86]:
compare = recordlinkage.Compare()

# add missing_value=pd.NA in compare.string if founded, marketcap, revenue, employees added
compare.string('name', 'name', label="name", threshold=0.60)
compare.string('country', 'country', label="country", threshold=0.60, missing_value=np.nan)
compare.string('sector', 'sector', label="sector", threshold=0.50, missing_value=np.nan)
# compare.add(CompareNumbers('founded', 'founded', label="founded"))
# compare.add(CompareNumbers('marketcap', 'marketcap', label="marketcap"))
# compare.add(CompareNumbers('revenue', 'revenue', label="revenue"))
# compare.add(CompareNumbers('employees', 'employees', label="employees"))
compare.string('links', 'links', method='jarowinkler', label="links", threshold=0.50, missing_value=np.nan)
compare.string('ceo', 'ceo', method='jarowinkler', label="ceo", threshold=0.60, missing_value=np.nan)

test_features = compare.compute(test_pairs, final, final)
test_features['score'] = test_features.loc[:, 'name':'ceo'].sum(axis=1)
test_features['null_values'] = test_features.loc[:, 'name':'ceo'].isnull().sum(axis=1)

In [ ]:
# uncomment if founded, marketcap, revenue, employees used
#for c in test_features.columns:
    #test_features[c] = test_features[c].replace(np.nan, 0)

In [87]:
col = len(final.columns) - 4
test_matches = test_features[test_features['score'] >= (col - test_features['null_values'])/2]
matches_test = test_matches
test_matches = test_matches.reset_index()

In [88]:
# toDrop = ['name', 'country', 'sector', 'founded', 'marketcap', 'revenue', 'employees', 'links', 'ceo', 'score', 'null_values']
toDrop = ['name', 'country', 'sector', 'links', 'ceo', 'score', 'null_values']
test_matches = test_matches.drop(toDrop, axis=1)
test_matches.head()

,level_0,level_1
0,138351,138351
1,138352,138352
2,138352,138385
3,138352,138386
4,138353,138353


In [89]:
test_matches = pd.MultiIndex.from_frame(test_matches)

In [93]:
dictionary_test = createDictionary(test_matches)

140000
140000
140000
140000
140000
140000
140000
140000
140000
140000
145000
145000
145000
145000
150000
150000
150000
155000
155000
155000
160000
160000
160000
160000
160000
160000
160000
160000
160000
160000
165000
165000
165000
165000
165000
165000
165000
165000
165000
165000
165000
165000
170000
170000
170000
170000
170000


In [94]:
dictionary_test

{138351: [138351],
 138352: [138352, 138385, 138386],
 138353: [138353,
  138354,
  138355,
  138356,
  138357,
  138358,
  138359,
  138360,
  138361,
  138362,
  138363,
  138364,
  138365,
  138366,
  138367,
  138368,
  138369,
  138370,
  138371,
  138372,
  138373,
  138374,
  138375,
  138382,
  138383,
  138435,
  138436,
  138437,
  138438,
  138439],
 138376: [138376],
 138377: [138377, 138378, 138379, 138380, 138381, 138401, 138403],
 138384: [138384],
 138387: [138387],
 138388: [138388,
  138389,
  138390,
  138391,
  138392,
  138393,
  138394,
  138395,
  138396,
  138397,
  138398,
  138399],
 138400: [138400],
 138402: [138402,
  138404,
  138405,
  138406,
  138410,
  138411,
  138413,
  138414,
  138415,
  138416,
  138417,
  138418,
  138419,
  138420],
 138407: [138407],
 138408: [138408, 138409, 138412, 138434],
 138421: [138421,
  138422,
  138423,
  138424,
  138425,
  138426,
  138427,
  138428,
  138429,
  138430,
  138431,
  138432,
  138433],
 138440: [13844

In [95]:
df_collapsed_test, indexesToDeleteTest = collapseMatches(final, dictionary_test)
df_collapsed_test.to_csv('./src/test_matches_collapsed.csv', index=False)

0
500
1000
1500
2000
2500
3000
3500
4000
4500
5000
5500
6000


In [99]:
test_features = test_features.fillna(0)
test_features.head()

name  country  sector  links  ceo  score
138350 138444   0.0      0.0     0.0    1.0  0.0    1.0
       138446   0.0      0.0     0.0    0.0  0.0    0.0
138351 138351   1.0      1.0     1.0    1.0  0.0    4.0
       138352   0.0      0.0     0.0    0.0  0.0    0.0
       138353   0.0      0.0     0.0    0.0  0.0    0.0

In [100]:
predictions = classifier.predict(test_features)

## Evaluation

In [101]:
# Confusion matrix
confusion_matrix = recordlinkage.confusion_matrix(test_matches, predictions, len(test_features))
print(confusion_matrix)


# Metrics
recall = recordlinkage.recall(test_matches, predictions)
print('\nRecall:', recall)
precision = recordlinkage.precision(test_matches, predictions)
print('Precision:', precision)
fscore = recordlinkage.fscore(confusion_matrix)
print('F-score:', fscore)
accuracy = recordlinkage.accuracy(test_matches, predictions, len(test_features))
print('Accuracy:', accuracy)

[[ 548297       0]
 [  46422 1282690]]

Recall: 1.0
Precision: 0.921942968023554
F-score: 0.9593863952910545
Accuracy: 0.9752733687757968


In [102]:
false_negatives = test_matches.difference(predictions)
false_negatives

MultiIndex([], )

In [103]:
try:
    fn_from_dfA = false_negatives[0][0]
    fn_from_dfB = false_negatives[0][1]

    display(final[final.index == fn_from_dfA])
    display(final[final.index == fn_from_dfB])
except:
    print("No False Negatives Present")

No False Negatives Present


In [104]:
try: 
    fn_from_dfA = false_negatives[1][0]
    fn_from_dfB = false_negatives[1][1]

    display(final[final.index == fn_from_dfA])
    display(final[final.index == fn_from_dfB])
except: 
    print("No False Negatives Present")

No False Negatives Present


## Linking tables

In [106]:
new_companies = pd.concat([df_collapsed_train, df_collapsed_test])

In [111]:
new_companies = new_companies.reset_index()

In [114]:
new_companies = new_companies.drop(columns=['index'])

In [115]:
new_companies

,name,country,sector,founded,marketcap,revenue,employees,links,ceo
0,ADDUS HOMECARE,USA,NaN,NaN,1690000000.0,NaN,NaN,NaN,NaN
1,#SINOB,GERMANY,RETAIL,2015.0,NaN,4.0,14.0,HTTPS://WWW.SINOB.DE/,NaN
2,(DIA) DISTRIBUIDORA INTERNACIONAL DE ALIMENTACIN,SPAIN,SUPERMARKETS FOOD RETAIL,NaN,870000000.0,NaN,NaN,NaN,NaN
3,(HLBANK) HONG LEONG BANK,MALAYSIA,BANKS FINANCIAL-SERVICES,NaN,9720000000.0,NaN,NaN,NaN,NaN
4,.NET IT SYSTEMHAUS,GERMANY,TECHNOLOGY,2014.0,NaN,4280960.0,83.0,HTTPS://WWW.NET-IT-SYSTEMHAUS.COM,NaN
...,...,...,...,...,...,...,...,...,...
32370,ŻABKA,POZNAŃ,FOOD RETAILERS & WHOLESALERS,1998.0,NaN,NaN,NaN,NaN,NaN
32371,ŻUBRÓWKA,BIAŁYSTOK,BREWERS,NaN,NaN,NaN,NaN,NaN,NaN
32372,ŽELEZNIČNÁ SPOLOČNOSŤ SLOVENSKO,BRATISLAVA,RAILROADS,2004.0,NaN,NaN,NaN,NaN,NaN
32373,ŽIA VALDA,VILNIUS,INVESTMENT SERVICES,1997.0,NaN,NaN,NaN,NaN,NaN


In [116]:
new_companies.to_csv('src/first_new_dataset.csv')

## Seconda passata di Matching

In [117]:
indexer = CustomIndex()
candidate_pairs2 = indexer.index(new_companies, new_companies)
pairs2 = candidate_pairs2

32375
0
5000
10000
15000
20000
25000
30000
192.93899703025818


In [118]:
pairs2

MultiIndex([(    0,     0),
            (    0,     7),
            (    0,    24),
            (    0,    28),
            (    0,    39),
            (    0,    42),
            (    0,    57),
            (    0,    80),
            (    0,    81),
            (    0,    97),
            ...
            (32367, 32369),
            (32368, 32368),
            (32369, 32369),
            (32369, 32370),
            (32369, 32373),
            (32370, 32370),
            (32370, 32371),
            (32371, 32371),
            (32372, 32372),
            (32373, 32373)],
           length=996080)

In [119]:
compare = recordlinkage.Compare()

# add missing_value=pd.NA in compare.string if founded, marketcap, revenue, employees added
compare.string('name', 'name', label="name", threshold=0.60)

features = compare.compute(pairs2, new_companies, new_companies)

In [121]:
features

name
0     0       1.0
      7       0.0
      24      0.0
      28      0.0
      39      0.0
...           ...
32370 32370   1.0
      32371   0.0
32371 32371   1.0
32372 32372   1.0
32373 32373   1.0

[996080 rows x 1 columns]

In [128]:
matches = features[features['name'] == 1]
matches = matches.drop(columns = ['name'])
matches = matches.reset_index()

In [132]:
matches

,level_0,level_1
0,0,0
1,1,1
2,2,2
3,3,3
4,4,4
...,...,...
39458,32369,32369
39459,32370,32370
39460,32371,32371
39461,32372,32372


In [133]:
matches_index = pd.MultiIndex.from_frame(matches)

In [134]:
matches_index

MultiIndex([(    0,     0),
            (    1,     1),
            (    2,     2),
            (    3,     3),
            (    4,     4),
            (    5,     5),
            (    6,     6),
            (    7,     7),
            (    8,     8),
            (    9,     9),
            ...
            (32364, 32364),
            (32365, 32365),
            (32366, 32366),
            (32367, 32367),
            (32368, 32368),
            (32369, 32369),
            (32370, 32370),
            (32371, 32371),
            (32372, 32372),
            (32373, 32373)],
           names=['level_0', 'level_1'], length=39463)

In [135]:
dictionary_final = createDictionary(matches_index)

0
5000
10000
15000
20000
25000
25000
25000
30000


In [136]:
collapsed_df_final, indexesToDeleteFinal = collapseMatches(new_companies, dictionary_final)
collapsed_df_final.to_csv('./src/final_matches_collapsed.csv', index=False)

0
500
1000
1500
2000
2500
3000
3500
4000
4500
5000
5500
6000
6500
7000
7500
8000
8500
9000
9500
10000
10500
11000
11500
12000
12500
13000
13500
14000
14500
15000
15500
16000
16500
17000
17500
18000
18500
19000
19500
20000
20500
21000
21500
22000
22500
23000
23500
24000
24500
25000
25500
26000
26500
27000
27500
28000
